# Advent of Code 2021

## Day 1

### Part 1

count the number of times a depth measurement increases

In [ ]:
let fileAsSequence fileName = 
    System.IO.File.ReadLines(fileName) 
    |> Seq.toArray

let depthMeasurements = 
    fileAsSequence "./Day 1.txt"
    |> Seq.map int

In [ ]:
let countIncreases list =
    list
    |> Seq.pairwise
    |> Seq.filter (fun ((a, b)) -> a < b)
    |> Seq.length

countIncreases depthMeasurements

1602

### Part 2

sums of a three-measurement sliding window

In [ ]:
depthMeasurements
|> Seq.windowed 3
|> Seq.map Seq.sum
|> countIncreases

1633

## Day 2

### Part 1

multiply your final horizontal position by your final depth?

In [ ]:
let asCartesianChange (direction, change) =
    match direction with
    | "forward" -> (change, 0)
    | "up" -> (0, -change)
    | "down" -> (0, change)
    | _ -> (0, 0)

let course =
    fileAsSequence "./Day 2.txt"
    |> Seq.map (fun line -> line.Split " ")
    |> Seq.map (fun pair -> (pair.[0], pair.[1] |> int))
    |> Seq.map asCartesianChange

In [ ]:
let moveSubmarine (horizontal, depth) (forwardChange, depthChange) =
    (horizontal + forwardChange, depth + depthChange)

let finalLocation =
    course
    |> Seq.reduce moveSubmarine

fst finalLocation * snd finalLocation

1524750

In [ ]:
var list = new List<int> { 1, 2, 3};
var sum = 0;
foreach (int item in list){
    var x = reducer(sum, item);
    sum = sum + item;
}

int reducer(int sum, int item) {
    return 0;
}

var total = list.Sum();

total

123

6

### Part 2


- down X increases your aim by X units.
- up X decreases your aim by X units.
- forward X does two things:
  - It increases your horizontal position by X units.
  - It increases your depth by your aim multiplied by X.

In [ ]:
let aimSubmarine (horizontal, depth) (forwardChange, aim) =
    (horizontal + forwardChange, depth + forwardChange * aim)

let aggregateAim (_, aim) (forwardChange, aimChange) =
    (forwardChange, aim + aimChange)

let finalLocation =
    course
    |> Seq.scan aggregateAim (0, 0)
    |> Seq.reduce aimSubmarine

fst finalLocation * snd finalLocation

0
0
9
13
13
15
22
13
18
18
18
20
20
20
25
27
18
25
25
31
37
36
38
38
38
38
37
37
36
36
36
33
26
30
31
31
31
28
28
30
30
28
27
22
22
13
13
9
13
13
6
7
8
16
16
16
9
8
12
6
12
12
15
24
30
22
22
31
24
31
31
39
33
40
44
37
37
37
37
37
42
51
59
64
61
64
70
76
76
78
78
83
81
81
82
85
85
87
87
93
93
93
93
102
106
114
121
124
118
125
125
116
124
124
124
126
122
122
115
117
117
117
122
118
121
124
129
131
136
136
136
141
137
143
143
137
132
136
136
136
143
134
130
121
121
129
137
139
147
155
149
149
158
158
158
166
161
161
161
167
167
167
167
162
164
164
158
152
152
149
149
149
158
163
159
152
144
144
144
151
156
156
162
162
158
152
154
154
154
145
140
144
139
141
141
132
133
133
142
142
147
149
145
137
145
152
145
146
146
146
148
149
149
156
150
150
150
158
158
150
150
146
146
152
152
152
152
152
152
156
156
156
156
156
159
159
153
144
148
154
154
163
161
169
175
176
176
178
181
181
181
186
179
177
177
181
185
185
189
185
185
189
189
189
189
192
196
196
196
196
190
190
190
190
190
190
190
191
1

1592426537

## Day 3

### Part 1

- gamma rate can be determined by finding the most common bit in the corresponding position
- epsilon rate can be determined by finding the most common bit in the corresponding position
- power consumption can then be found by multiplying the gamma rate by the epsilon rate

In [ ]:
let charToBit bit = if bit = '1' then 1 else 0

let diagnostics =
    fileAsSequence "./Day 3.txt"
    |> Seq.map Seq.toList
    |> Seq.map (Seq.map charToBit)

let mostCommonBit = Seq.groupBy id >> Seq.sortByDescending fst >> Seq.maxBy (snd >> Seq.length) >> fst
let consumptionRate = Seq.transpose >> Seq.map mostCommonBit

let decode adapter = Seq.fold (fun rate bit -> (rate <<< 1) + (adapter bit)) 0

let power report =
    let consumption = consumptionRate report
    let bitMask = ((1 <<< Seq.length consumption) - 1)
    let gamma = decode id consumption
    let epsilon = ~~~gamma &&& bitMask
    gamma * epsilon

power diagnostics

1997414

### Part 2

- verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

In [ ]:
let negate bit = ~~~bit &&& 1
let filteredBy orderBy column = Seq.transpose >> Seq.item column >> orderBy
let leastCommonBit = mostCommonBit >> negate

let lifeSupport commonBitFilter report =
    let filterByCommonBit bit data =
        let commonBit = commonBitFilter bit data
        Seq.filter (fun row -> Seq.item bit row = commonBit) data

    let rec getRating bit data =
        if Seq.length data = 1
        then data
        else getRating (bit + 1) (filterByCommonBit bit data)
    
    (getRating 0 report) |> Seq.head |> (decode id)

let oxygenRating = lifeSupport (filteredBy mostCommonBit) diagnostics
let scrubberRating = lifeSupport (filteredBy leastCommonBit) diagnostics

oxygenRating * scrubberRating

1032597

## Day 4

### Part 1

Bingo!

In [ ]:
let bingo =
    fileAsSequence "./Day 4.txt"

let split c (line:string) =
    line.Split c

let draws =
    bingo
    |> Seq.head
    |> (fun line -> (line.Split ','))
    |> Seq.map int

let isEmpty line = line = ""

let parseBoard = (fun (x:string) -> x.Trim().Replace("  ", " ").Split(" ") |> (Seq.map int))

let getBoards boards =
    let rows = 
        boards
        |> Seq.skip 1
        |> Seq.filter (isEmpty >> not)
        |> Seq.map parseBoard
    Seq.chunkBySize (Seq.length (Seq.head rows)) rows

let boards = getBoards bingo
boards

index,value
0,"[ [ 22, 59, 7, 10, 6 ], [ 33, 36, 96, 55, 23 ], [ 13, 85, 18, 29, 28 ], [ 75, 46, 83, 73, 58 ], [ 34, 40, 87, 56, 98 ] ]"
1,"[ [ 73, 96, 47, 14, 10 ], [ 28, 11, 79, 84, 20 ], [ 74, 30, 0, 59, 71 ], [ 80, 93, 42, 22, 17 ], [ 44, 2, 81, 29, 15 ] ]"
2,"[ [ 73, 32, 37, 93, 39 ], [ 2, 87, 38, 99, 97 ], [ 15, 12, 42, 49, 33 ], [ 9, 23, 25, 94, 19 ], [ 57, 4, 18, 70, 79 ] ]"
3,"[ [ 43, 79, 59, 71, 78 ], [ 51, 97, 37, 28, 26 ], [ 46, 21, 84, 8, 86 ], [ 96, 30, 17, 34, 49 ], [ 55, 90, 99, 75, 61 ] ]"
4,"[ [ 43, 86, 35, 51, 24 ], [ 16, 25, 29, 21, 3 ], [ 2, 13, 4, 36, 54 ], [ 89, 27, 8, 85, 34 ], [ 96, 59, 65, 14, 56 ] ]"
5,"[ [ 5, 57, 43, 4, 1 ], [ 86, 80, 67, 30, 20 ], [ 22, 16, 99, 0, 14 ], [ 40, 25, 59, 91, 54 ], [ 82, 95, 96, 37, 24 ] ]"
6,"[ [ 97, 20, 87, 66, 21 ], [ 25, 40, 9, 78, 89 ], [ 52, 75, 95, 63, 62 ], [ 32, 43, 13, 47, 69 ], [ 53, 48, 56, 29, 4 ] ]"
7,"[ [ 24, 4, 65, 12, 8 ], [ 76, 3, 81, 99, 49 ], [ 45, 33, 31, 66, 34 ], [ 17, 94, 75, 35, 88 ], [ 64, 10, 7, 93, 95 ] ]"
8,"[ [ 88, 20, 99, 15, 83 ], [ 81, 40, 5, 6, 98 ], [ 49, 74, 18, 27, 9 ], [ 43, 69, 28, 37, 71 ], [ 87, 82, 90, 14, 47 ] ]"
9,"[ [ 21, 97, 92, 42, 60 ], [ 11, 65, 98, 95, 29 ], [ 2, 49, 75, 20, 74 ], [ 56, 40, 78, 66, 81 ], [ 68, 4, 46, 77, 61 ] ]"


In [ ]:
let membership values = Seq.map (fun x -> if (Seq.contains x values) then 1 else 0)
let isWin values board = 
    Seq.concat [board; Seq.transpose board] 
    |> Seq.map (membership values)
    |> Seq.map Seq.sum
    |> Seq.contains (board |> Seq.head |> Seq.length)

let score drawn board = (board |> Seq.concat |> Seq.filter (fun x -> not (Seq.contains x drawn)) |> Seq.sum) * (Seq.last drawn)

let rec playBingo draws boards round =
    let drawn = Seq.take round draws
    let result = Seq.tryFind (isWin drawn) boards
    match (result) with
    | None -> playBingo draws boards (round + 1)
    | Some board -> score drawn board

playBingo draws boards 5

71708

### Part 2

- figure out which board will win last

In [ ]:
let rec findLastWinningBoard draws boards round =
    let drawn = List.take round draws
    let remainingBoards = List.filter (isWin drawn >> not) boards
    if (List.length remainingBoards = 0)
    then score drawn (List.head boards)
    else findLastWinningBoard draws remainingBoards (round + 1)

findLastWinningBoard (draws |> Seq.toList) (boards |> Seq.toList) 5

34726

## Day 5

### Part 1

- count number of points where lines overlap

In [ ]:
let parseCoordinates (line: string) = line.Replace(" -> ", ",").Split(',') |> Seq.map int |> Seq.chunkBySize 2

let coordinates =
    fileAsSequence "./Test.txt"
    |> Seq.map parseCoordinates
    |> Seq.map Seq.toList

let isLine coordinates =
    match coordinates with
    | [[|x1;y1|];[|x2;y2|]] -> x1 = x2 || y1 = y2
    | _ -> false

let findAllLines = Seq.filter isLine
let dimensions lines = lines |> Seq.map Seq.transpose |> Seq.transpose |> Seq.map Seq.concat |> Seq.map Seq.max |> Seq.pairwise |> Seq.head

let lines = findAllLines coordinates
lines

index,value
0,"[ [ 0, 9 ], [ 5, 9 ] ]"
1,"[ [ 9, 4 ], [ 3, 4 ] ]"
2,"[ [ 2, 2 ], [ 2, 1 ] ]"
3,"[ [ 7, 0 ], [ 7, 4 ] ]"
4,"[ [ 0, 9 ], [ 2, 9 ] ]"
5,"[ [ 3, 4 ], [ 1, 4 ] ]"
